In [21]:
import torch
import torch.nn as nn
import sys
import nltk
import torch.nn.functional as F
import torch
import torch.nn as nn
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import torch
nltk.download('punkt')
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
sys.path.insert(0, '..')
import pickle
from torch.autograd import grad as torch_grad



word_embedding_dim = 50
z_size = 100
output_shape = (1, 105, 8)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gxb18167\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [22]:
print(torch.__version__)
print("GPU Available:", torch.cuda.is_available())

if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = "cpu"

2.0.1
GPU Available: False


In [23]:
class Generator(nn.Module):
    def __init__(self, noise_dim, word_embedding_dim, output_shape):
        super(Generator, self).__init__()

        self.noise_dim = noise_dim
        self.word_embedding_dim = word_embedding_dim

        # Define the layers of your generator
        self.fc_noise = nn.Linear(noise_dim, 105*8)  # Increase the size for more complexity
        self.fc_word_embedding = nn.Linear(word_embedding_dim, 105*8)  # Increase the size for more complexity
        self.conv1 = nn.Conv2d(2, 128, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(128)
        self.relu = nn.LeakyReLU(0.2)

        self.conv2 = nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)

        self.conv3 = nn.Conv2d(64, 1, kernel_size=3, stride=1, padding=1)
        self.tanh = nn.Tanh()

    def forward(self, noise, word_embedding):
        # Process noise
        noise = self.fc_noise(noise)
        noise = noise.view(noise.size(0), 1, 105,8)  # Adjust the size to match conv1

        # Process word embedding
        word_embedding = self.fc_word_embedding(word_embedding.to(device))
        word_embedding = word_embedding.view(word_embedding.size(0), 1, 105, 8)  # Adjust the size to match conv1

        # Concatenate noise and word embedding
        combined_input = torch.cat([noise, word_embedding], dim=1)

        # Upsample and generate the output
        z = self.conv1(combined_input)
        z = self.bn1(z)
        z = self.relu(z)

        z = self.conv2(z)
        z = self.bn2(z)
        z = self.relu(z)

        z = self.conv3(z)
        z = self.tanh(z)

        return z

In [76]:
# Assuming gen_model is the generator model you used during training
gen_model = Generator(z_size, word_embedding_dim, output_shape)  # Replace with your actual generator model class
checkpoint = torch.load(r"I:\Science\CIS-YASHMOSH\niallmcguire\WGAN_Text_2.0\Textual_WGAN_GP_checkpoint_epoch_100.pt",
                        map_location=torch.device('cpu'))  # Load the final model checkpoint


In [77]:
# Load the model's state_dict onto the CPU
gen_model.load_state_dict(checkpoint['gen_model_state_dict'])

# Set the model to evaluation mode
gen_model.eval()

Generator(
  (fc_noise): Linear(in_features=100, out_features=840, bias=True)
  (fc_word_embedding): Linear(in_features=50, out_features=840, bias=True)
  (conv1): Conv2d(2, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): LeakyReLU(negative_slope=0.2)
  (conv2): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (tanh): Tanh()
)

In [78]:
def create_noise(batch_size, z_size, mode_z):
    if mode_z == 'uniform':
        input_z = torch.rand(batch_size, z_size)*2 - 1
    elif mode_z == 'normal':
        input_z = torch.randn(batch_size, z_size)
    return input_z

In [79]:
def generate_samples(g_model, input_z, input_t):
    # Create random noise as input to the generator
    # Generate samples using the generator model
    with torch.no_grad():
        g_output = g_model(input_z, input_t)

    return g_output.cpu().numpy()

In [80]:
import pickle

# To load the lists from the file:
with open(r"C:\Users\gxb18167\PycharmProjects\EEG-To-Text\SIGIR_Development\EEG-GAN\EEG_Text_Pairs.pkl", 'rb') as file:
    EEG_word_level_embeddings = pickle.load(file)
    EEG_word_level_labels = pickle.load(file)

In [81]:
def create_word_label_embeddings(Word_Labels_List):
    tokenized_words = []
    for i in range(len(Word_Labels_List)):
        tokenized_words.append([Word_Labels_List[i]])
    model = Word2Vec(sentences=tokenized_words, vector_size=word_embedding_dim, window=5, min_count=1, workers=4)
    word_embeddings = {word: model.wv[word] for word in model.wv.index_to_key}
    print("Number of word embeddings:", len(word_embeddings))
    #word, embedding = list(word_embeddings.items())[10]
    #print(f"Word: {word}, Embedding: {embedding}")

    Embedded_Word_labels = []
    for word in EEG_word_level_labels:
        Embedded_Word_labels.append(word_embeddings[word])

    return Embedded_Word_labels, word_embeddings

def create_dataloader(EEG_word_level_embeddings, Embedded_Word_labels):
    #EEG_word_level_embeddings_normalize = (EEG_word_level_embeddings - np.mean(EEG_word_level_embeddings)) / np.std(EEG_word_level_embeddings)

    # Assuming EEG_synthetic is the generated synthetic EEG data
    #EEG_synthetic_denormalized = (EEG_synthetic * np.max(np.abs(EEG_word_level_embeddings))) + np.mean(EEG_word_level_embeddings)


    EEG_word_level_embeddings_normalize = (EEG_word_level_embeddings - np.mean(EEG_word_level_embeddings)) / np.max(np.abs(EEG_word_level_embeddings))


    float_tensor = torch.tensor(EEG_word_level_embeddings_normalize, dtype=torch.float)
    float_tensor = float_tensor.unsqueeze(1)

    #print(EEG_word_level_embeddings_normalize)
    # Calculate mean and standard deviation
    print(torch.isnan(float_tensor).any())

    train_data = []
    for i in range(len(float_tensor)):
       train_data.append([float_tensor[i], Embedded_Word_labels[i]])
    trainloader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=64)
    return trainloader

In [82]:
Embedded_Word_labels, word_embeddings = create_word_label_embeddings(EEG_word_level_labels)
trainloader = create_dataloader(EEG_word_level_embeddings, Embedded_Word_labels)

Number of word embeddings: 2397
tensor(False)


In [83]:
i1, l1 = next(iter(trainloader))
word_embedding = l1
input_z = create_noise(64, 100, "uniform")

In [84]:
g_output = generate_samples(gen_model, input_z, word_embedding)

In [85]:
# Assuming EEG_synthetic is the generated synthetic EEG data
EEG_synthetic_denormalized = (g_output * np.max(np.abs(EEG_word_level_embeddings))) + np.mean(EEG_word_level_embeddings)


In [87]:
EEG_synthetic_denormalized[0][0]

array([[ 2.1955438 ,  2.6450155 ,  3.7999616 ,  4.759287  ,  3.892948  ,
         2.2888477 ,  2.2012181 ,  2.629305  ],
       [ 4.404852  ,  4.1778    ,  2.260309  ,  2.3867383 ,  3.905774  ,
         3.23086   ,  3.83308   ,  4.1817355 ],
       [ 1.392333  ,  2.6844435 ,  3.6532493 ,  1.8740801 ,  4.0452604 ,
         3.8101861 ,  4.371214  ,  2.5289278 ],
       [ 0.90442085,  1.700145  ,  2.5674434 ,  2.8517385 ,  2.604467  ,
         2.281059  ,  2.044942  ,  1.3113654 ],
       [ 1.6170524 ,  1.5853921 ,  2.0599527 ,  2.508909  ,  2.165251  ,
         1.2945838 ,  1.2241237 ,  1.5874115 ],
       [ 1.1482277 ,  1.5218067 ,  1.3293641 ,  1.0194237 ,  0.5397637 ,
         0.6863346 ,  1.6726389 ,  1.6313049 ],
       [ 1.7355064 ,  1.8320003 ,  1.8136117 ,  1.5623896 ,  1.7409168 ,
         0.78047496,  1.9885063 ,  2.4840758 ],
       [ 2.8789015 ,  2.8320374 ,  1.5604337 ,  1.5806464 ,  4.268983  ,
         4.040824  ,  1.9493328 ,  2.089771  ],
       [ 1.5232179 ,  1.4098688 